# Quantum Solace - Educational quantum game for Epitech courses

This project is a pedagogy based game when you can play and try to make circuits with quantum games in order to familar yourself with quantum's concept.

We are three students from Epitech, David Zeng, Benjamin Audet, Patrick BUI, and we choose to make this project for our courses in order to help any new learner about Quantum.

# Context

This following snippet is explaining how we are using the circuit built by the user on the Web Interface to an exploitable working circuit running under Qiskit. This allow to test his circuit and check if the result is right compared to the expected output.
At the end, We feedback the user with the plot of his circuit.

### Imports

Essential tools to generate qiskit circuits

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.visualization import plot_histogram
import circuit as C
import base64
import io

Circuit = C.Circuit
Gate = C.Gate

### Mapping of each gate to the corresponding Qiskit action

We handle the following gates and/or actions:
  - Barrier
  - H gate
  - CX gate
  - Z gate
  - X gate

In [0]:
def getActionParams(qc: QuantumCircuit, gate: Gate):
    actions = {
        "barrier": qc.barrier,
        "H": qc.h,
        "CX": qc.cx,
        "Z": qc.z,
        "X": qc.x
    }
    action = actions.get(gate.name)
    source = gate.option == "control" or (gate.option != "target" and action != None)
    target = gate.option == "target"
    return action, source, target

### Run Simulation and generate Qiskit circuit

Convert user gates to qiskit circuit and run it on local quantum simulator.\
We generate the state vector for the bloch sphere.\
We measure each register at the end.\
The circuit is run with 1024 shots.

In [0]:
def runSimulation(circuit: Circuit):
    nbRegisters = len(circuit.registers)
    nbGates = len(circuit.registers[0].gates)

    qc = QuantumCircuit(nbRegisters, nbRegisters)

    for i_gate in range(nbGates):
        source = -1
        target = -1
        action = None
        for i_reg in range(len(circuit.registers)):
            register = circuit.registers[i_reg]
            gate = register.gates[i_gate]
            if gate != None:
                a, s, t = getActionParams(qc, gate)
                if a != None:
                    action = a
                if s == True:
                    source = i_reg
                if t == True:
                    target = i_reg
        if action != None and target == -1:
            action(source)
        elif action != None:
            action(source, target)

    job = execute(qc, Aer.get_backend("statevector_simulator"))
    statevector = job.result().get_statevector()

    for i in range(nbRegisters):
        qc.measure(i, i)

    backend = Aer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=1024)
    result = job.result()
    count = result.get_counts(qc)
    return count, statevector

### Generate Plot from the circuit
Generate a plot from the qiskit circuit result that will be sent and displayed on the user interface.\
For the level 1 circuit, we generate the bloch sphere of the qubit.\
For the others, we generate the histogram plot.

In [0]:
def getBase64Plot(count, statevector, level):
    ioBytes = io.BytesIO()
    if level == 1:
        fig = plot_bloch_multivector(statevector)
    else:
        fig = plot_histogram(count)
    fig.savefig(ioBytes, format = 'png')
    ioBytes.seek(0)
    encodedPlot = base64.b64encode(ioBytes.read())
    return encodedPlot.decode('utf-8')